In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mohmmadayaan_complete_dataset_path = kagglehub.dataset_download('mohmmadayaan/complete-dataset')

print('Data source import complete.')


100%|██████████| 7.80M/7.80M [00:00<00:00, 68.4MB/s]

Extracting files...


Data source import complete.


In [6]:
pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.4 MB/s eta 0:00:00


In [8]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.7 MB/s eta 0:00:00


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.metrics import roc_auc_score
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, zscore
from statsmodels.stats.weightstats import ztest
from scipy.stats import chi2_contingency

In [11]:
df = pd.read_csv('/content/complete_dataset.csv')

In [12]:
print(df)

              id  CustomerId         Surname  CreditScore Geography  Gender  \
0            0.0  15674932.0  Okwudilichukwu        668.0    France    Male   
1            1.0  15749177.0   Okwudiliolisa        627.0    France    Male   
2            2.0  15694510.0           Hsueh        678.0    France    Male   
3            3.0  15741417.0             Kao        581.0    France    Male   
4            4.0  15766172.0       Chiemenam        716.0     Spain    Male   
...          ...         ...             ...          ...       ...     ...   
145064  145064.0  15795738.0             Hs?        521.0   Germany  Female   
145065  145065.0  15772383.0      Tretiakova        636.0    France    Male   
145066  145066.0  15619238.0           Hsiao        696.0   Germany  Female   
145067  145067.0  15661532.0           Rizzo        645.0    France  Female   
145068  145068.0  15592717.0          Goliwe        708.0     Spain  Female   

         Age  Tenure    Balance  NumOfProducts  Has

In [13]:
df = df.drop_duplicates()

In [14]:
def fill_missing_values(df):

    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].apply(lambda col: col.fillna(col.mean()))

    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].apply(lambda col: col.fillna(col.mode()[0]))

    return df

df = fill_missing_values(df)


In [15]:
num_col = df.select_dtypes(include=['number']).columns
print(num_col)

Index(['id', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')


In [16]:
import numpy as np

def normalize_outliers(df):
    for col in df.select_dtypes(include=['float64', 'int64']).columns:


        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df[col] = np.where(df[col] < lower_bound, df[col].quantile(0.01), df[col])
        df[col] = np.where(df[col] > upper_bound, df[col].quantile(0.99), df[col])

    return df

df = normalize_outliers(df)


In [10]:
import pandas as pd
import numpy as np
from sklearn.random_projection import GaussianRandomProjection
from sklearn.preprocessing import OneHotEncoder

# Load data
df = pd.read_csv('/content/complete_dataset.csv')
df = df.drop_duplicates()
def normalize_outliers(df):
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df.loc[df[col] < lower_bound, col] = df[col].quantile(0.01)
        df.loc[df[col] > upper_bound, col] = df[col].quantile(0.99)
    return df
# Fill missing values function
def fill_missing_values(df):
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].apply(lambda col: col.fillna(col.mean()))

    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].apply(lambda col: col.fillna(col.mode()[0]))
    return df

# Apply normalization and fill missing values
df = normalize_outliers(df)
df = fill_missing_values(df)

# Drop unnecessary columns
columns_to_drop = ['id', 'CustomerId', 'Surname']  # Adjust as needed
df = df.drop(columns=columns_to_drop)

# Convert categorical columns to one-hot encoding
categorical_columns = ['Geography', 'Gender']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_categorical = encoder.fit_transform(df[categorical_columns])

# Combine numerical and categorical data for projection
numerical_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
                     'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
numerical_data = df[numerical_columns].values
combined_data = np.hstack([numerical_data, encoded_categorical])

# Random Projection
n_components = min(10, combined_data.shape[1])  # Ensure n_components <= features
projector = GaussianRandomProjection(n_components=n_components, random_state=42)
projected_data = projector.fit_transform(combined_data)

# Show the projected data
print("Shape of projected data:", projected_data.shape)


Shape of projected data: (275058, 10)


In [15]:


# # Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Random Forest Accuracy on Random Projected Data: {accuracy:.4f}")


Random Forest Accuracy on Random Projected Data: 0.7871


In [16]:

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"XGBoost Accuracy on Random Projected Data: {accuracy:.4f}")


XGBoost Accuracy on Random Projected Data: 0.8113


In [17]:


# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

# Make predictions
y_pred = dt_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Decision Tree Accuracy on Random Projected Data: {accuracy:.4f}")


Decision Tree Accuracy on Random Projected Data: 0.7167


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Suppress convergence warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.linear_model')

# Initialize Logistic Regression model
lr_model = LogisticRegression(random_state=42, max_iter=2000)  # Increased max_iter for convergence

# Train the model
lr_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = lr_model.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Logistic Regression Accuracy on Random Projected Data: {accuracy:.4f}")


Logistic Regression Accuracy on Random Projected Data: 0.8114
